# Problem 4
Showing that the determinant of the electromagnetic field tensor $F_{\mu\nu}$ is proportional to $(\vec{E}\cdot\vec{B})^2$ using the 4-dimensional Levi Civita symbol where
$$
    \det F \epsilon_{j_1... j_4} = \sum_{i_1 ... i_4} \epsilon_{i_1 ... i_4}F_{i_1j_1}... F_{i_4j_4}
$$

## Step 1: Finding all permutations of 0123
We begin by writing down all the possible permutations of 0123 with the goal of then reducing these to only the ones that contribute to the sum 

In [1]:
from itertools import permutations

In [2]:
perm = permutations([0,1,2,3])

### Storing permutations in a list

In [3]:
perm_list = [] 
for item in list(perm):
    perm_list.append(item)

In [4]:
# all 24 permutations
perm_list

[(0, 1, 2, 3),
 (0, 1, 3, 2),
 (0, 2, 1, 3),
 (0, 2, 3, 1),
 (0, 3, 1, 2),
 (0, 3, 2, 1),
 (1, 0, 2, 3),
 (1, 0, 3, 2),
 (1, 2, 0, 3),
 (1, 2, 3, 0),
 (1, 3, 0, 2),
 (1, 3, 2, 0),
 (2, 0, 1, 3),
 (2, 0, 3, 1),
 (2, 1, 0, 3),
 (2, 1, 3, 0),
 (2, 3, 0, 1),
 (2, 3, 1, 0),
 (3, 0, 1, 2),
 (3, 0, 2, 1),
 (3, 1, 0, 2),
 (3, 1, 2, 0),
 (3, 2, 0, 1),
 (3, 2, 1, 0)]

## Step 2: Remove ones that would yield zero
The ones that start with 0, have second element be 1, third element be 2, and fourth element be 3, wouldn't contribute to the sum since it would correspond to diagonal $F$ matrix elements which are zero. That is, we want to remove the ones that look like
$$
    \epsilon_{0jkl} \qquad \epsilon_{i1kl} \qquad \epsilon_{ij2l} \qquad \epsilon_{ijk3}
$$

In [5]:
perm_list = [x for x in perm_list if not x[0] == 0]
perm_list = [x for x in perm_list if not x[1] == 1]
perm_list = [x for x in perm_list if not x[2] == 2]
perm_list = [x for x in perm_list if not x[3] == 3]

In [6]:
perm_list

[(1, 0, 3, 2),
 (1, 2, 3, 0),
 (1, 3, 0, 2),
 (2, 0, 3, 1),
 (2, 3, 0, 1),
 (2, 3, 1, 0),
 (3, 0, 1, 2),
 (3, 2, 0, 1),
 (3, 2, 1, 0)]

## Step 3: Symbolic Manipulation
Use `sympy` to write symbolic expressions for the various components of the electric and magnetic fields and construct the electromagnetic field tensor $F$
$$
    F = \begin{pmatrix}
    0 &-E_x &-E_y &-E_z\\
    E_x &0 &-B_z &B_y\\
    E_y &B_z &0 &-B_x\\
    E_z &-B_y &B_x &0
    \end{pmatrix} 
$$

#### Import `sympy` a python library for symbolic calculations

In [7]:
import sympy as sym

#### Declare symbolic symbol variables 

In [8]:
Ex, Ey, Ez, Bx, By, Bz = sym.symbols('E_x, E_y, E_z, B_x, B_y, B_z')

#### Construct F

In [9]:
F = [[0, -Ex, -Ey, -Ez], [Ex, 0, -Bz, By], [Ey, Bz, 0, -Bx], [Ez, -By, Bx, 0]]

## Step 4: Next write a function that takes in a symbol and returns the combination of $F$ matrix elements 
Notice that in our sum we have terms of the form 
$$
    \epsilon_{ijkl}F_{0i}F_{1j}F_{2k}F_{3l}
$$
For a specific $ijkl$ we would like a function that returns the corresponding $F_{0i}F_{1j}F_{2k}F_{3l}$. 

In [10]:
def F_matrix_combination(symbol, matrix):
    """ takes a levi-civita symbol permutation
        and returns corresponding combination of F_matrix 
        ex. input: epsilon_{1234} -> return: F_{11}F_{22}F_{33}F_{44}
    """
    product = 1
    for i in range(4):
        index = symbol[i]
        product = product * matrix[i][index]
    return product 

In [11]:
F_matrix_combination(perm_list[2], F)

-B_x*B_y*E_x*E_y

## Step 5: Calculate determinant 
We're almost there! Before we can write down the determinant, we need a function that checks for the parity of the given permutation and returns +1 for even permutations and -1 for odd permutations. This function below does this. 

In [14]:
def parity(p):
    """ 
        checks parity of permutation 
        - returns true for even parity 
        - returns false for odd parity 
    """
    boolean = sum(
        1 for (x,px) in enumerate(p)
          for (y,py) in enumerate(p)
          if x<y and px>py
        )%2==0
    
    if boolean == True:
        return 1 
    else:
        return -1 

### Defining the determinant function
1. Note that this only works for the very limited case of our electromagnetic field tensor  

In [15]:
def determinant(symbol_list, matrix):
    """ calculates determinant for this very specific case """
    summation = 0
    for symbol in symbol_list:
        summation += parity(symbol)*F_matrix_combination(symbol, matrix)
    return summation 

In [16]:
determinant(perm_list, F)

B_x**2*E_x**2 + 2*B_x*B_y*E_x*E_y + 2*B_x*B_z*E_x*E_z + B_y**2*E_y**2 + 2*B_y*B_z*E_y*E_z + B_z**2*E_z**2

### Verifying 
Checking to see if our result matches. Remember, we want to get that this is proportional to 
$$
    (\vec{E}\cdot\vec{B})^2 
$$

In [17]:
E = [Ex, Ey, Ez]
B = [Bx, By, Bz]

In [18]:
import numpy as np 
sym.expand(np.dot(E,B)**2)

B_x**2*E_x**2 + 2*B_x*B_y*E_x*E_y + 2*B_x*B_z*E_x*E_z + B_y**2*E_y**2 + 2*B_y*B_z*E_y*E_z + B_z**2*E_z**2

### Extra bit:
1. We can take this `sympy` output and have it printed in LaTeX which is very useful for copying and pasting for writing up stuff lol

In [21]:
from sympy import print_latex
print_latex(sym.expand(np.dot(E,B)**2))

B_{x}^{2} E_{x}^{2} + 2 B_{x} B_{y} E_{x} E_{y} + 2 B_{x} B_{z} E_{x} E_{z} + B_{y}^{2} E_{y}^{2} + 2 B_{y} B_{z} E_{y} E_{z} + B_{z}^{2} E_{z}^{2}
